In [26]:
import dill
dill.dump_session("EXP5.db")

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



In [2]:
from keras.models import Sequential
from keras.layers import Conv1D, ConvLSTM2D, MaxPooling1D, Flatten, Dense, Reshape,TimeDistributed,Conv2D,BatchNormalization,MaxPooling2D,ConvLSTM1D,LSTM



In [3]:
import tensorflow as tf
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)


In [4]:
# Read CSV File
df = pd.read_csv(r"C:\Users\mujta\OneDrive\Documents\GitHub\F20PA-DriverDrowsinessDetection\Experiments\mergedEMYA_min_max_normEMYA.csv")

In [5]:
# Select frames at 5 fps
df = df.iloc[::2]


In [6]:
df.drop(columns=["Unnamed: 0","Part_ID"],inplace=True)

In [7]:
df

,Label,EAR,MAR,MOE,Angle
0,0,0.695510,0.616864,0.391060,0.531289
2,0,0.700934,0.642625,0.404756,0.598722
4,0,0.707608,0.535723,0.335285,0.558230
6,0,0.752346,0.612585,0.364306,0.584936
8,0,0.692796,0.607403,0.386318,0.524120
...,...,...,...,...,...
422990,10,0.491076,0.604703,0.363197,0.611701
422992,10,0.523784,0.654568,0.354916,0.607305
422994,10,0.571002,0.590067,0.304906,0.586004
422996,10,0.471807,0.654889,0.392917,0.601479


In [8]:
int(df.shape[0]/5)

42300

In [9]:

from sklearn.metrics import precision_recall_fscore_support
# Function to get precision, recall, and F1 scores
def getPerformanceMetrics(model,X_test,y_test):

    y_pred_proba = model.predict(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    # Compute precision, recall, and F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(y_test.flatten(), y_pred, average=None)
    # Lists of precision , recall, and F1 scores
    p_list = []
    r_list = []
    f_list = []
    # Print precision, recall, and F1 score for each class
    for i in range(len(precision)):
        p_list.append(precision[i])
        r_list.append(recall[i])
        f_list.append(f1[i])
        print(f"Class {i}:")
        print(f"  Precision: {precision[i]}")
        print(f"  Recall: {recall[i]}")
        print(f"  F1 Score: {f1[i]}")
    # Print Average values
    print(f"Avg Precision:{np.average(p_list)}")
    print(f"Avg Recall:{np.average(r_list)}")
    print(f"Avg F1 Score:{np.average(f_list)}")


In [10]:
# Function to set parameters and train model.
# Returns model, and test sets.
def compileModel(data,timestep,test_size,num_epochs):
    # Separate labels
    y=data["Label"]
    data = data.drop(columns=['Label'])
    # Reshape according to timestep
    df_reshaped = np.array(data).reshape(int(data.shape[0]/timestep),timestep,3)
    y = np.array(y)
    y_reshaped  = []
    for i in range(0, len(y), timestep):
        y_reshaped.append([y[i]])
    # Create test and  train sets.
    X_train, X_test, y_train, y_test = train_test_split(df_reshaped, y_reshaped, test_size=test_size, random_state=42)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train= np.array(y_train)
    y_test = np.array(y_test)
    # Set labels as  0,1,2
    for i in range(0,len(y_train)):
        y_train[i]=int(y_train[i]/5)
    for i in range(0,len(y_test)):
        y_test[i]=int(y_test[i]/5)
    # Define model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(timestep, 3)))
    model.add(MaxPooling1D(pool_size=2))

    model.add(LSTM(64, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='softmax'))  

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=num_epochs, batch_size=32, validation_split=0.2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy:", accuracy)
    getPerformanceMetrics(model,X_test,y_test)
    return model, X_test, y_test






## Increase timestep 5,10,20

In [11]:
modelA , X_testA, y_testA = compileModel(data=df,
                                         timestep=5,
                                         test_size=0.2,
                                         num_epochs=50)




Epoch 1/50


846/846 [==============================] - 4s 3ms/step - loss: 1.0596 - accuracy: 0.4137 - val_loss: 1.0401 - val_accuracy: 0.4112
Epoch 2/50
846/846 [==============================] - 2s 2ms/step - loss: 1.0288 - accuracy: 0.4414 - val_loss: 1.0183 - val_accuracy: 0.4374
Epoch 3/50
846/846 [==============================] - 2s 3ms/step - loss: 1.0005 - accuracy: 0.4849 - val_loss: 0.9854 - val_accuracy: 0.4886
Epoch 4/50
846/846 [==============================] - 2s 2ms/step - loss: 0.9716 - accuracy: 0.5186 - val_loss: 0.9715 - val_accuracy: 0.5121
Epoch 5/50
846/846 [==============================] - 2s 2ms/step - loss: 0.9471 - accuracy: 0.5372 - val_loss: 0.9566 - val_accuracy: 0.5325
Epoch 6/50
846/846 [==============================] - 2s 3ms/step - loss: 0.9238 - accuracy: 0.5561 - val_loss: 0.9344 - val_accuracy: 0.5525
Epoch 7/50
846/846 [==============================] - 2s 3ms/step - loss: 0.9070 - accuracy: 0.5716 - val_loss: 0.8985 - val_accuracy: 0.5725
E

In [12]:
modelA.save("modelA.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
modelB , X_testB, y_testB = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.2,
                                         num_epochs=50)

Epoch 1/50
423/423 [==============================] - 6s 6ms/step - loss: 1.0567 - accuracy: 0.4227 - val_loss: 1.0347 - val_accuracy: 0.4477
Epoch 2/50
423/423 [==============================] - 2s 5ms/step - loss: 1.0286 - accuracy: 0.4590 - val_loss: 1.0024 - val_accuracy: 0.4967
Epoch 3/50
423/423 [==============================] - 2s 5ms/step - loss: 1.0026 - accuracy: 0.4826 - val_loss: 0.9848 - val_accuracy: 0.5118
Epoch 4/50
423/423 [==============================] - 2s 5ms/step - loss: 0.9780 - accuracy: 0.5103 - val_loss: 0.9481 - val_accuracy: 0.5298
Epoch 5/50
423/423 [==============================] - 2s 5ms/step - loss: 0.9473 - accuracy: 0.5383 - val_loss: 0.9338 - val_accuracy: 0.5473
Epoch 6/50
423/423 [==============================] - 2s 5ms/step - loss: 0.9182 - accuracy: 0.5567 - val_loss: 0.9036 - val_accuracy: 0.5694
Epoch 7/50
423/423 [==============================] - 2s 5ms/step - loss: 0.8765 - accuracy: 0.5871 - val_loss: 0.8380 - val_accuracy: 0.6079
Epoch 

In [14]:
modelB.save("modelB.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
modelC , X_testC, y_testC = compileModel(data=df,
                                         timestep=20,
                                         test_size=0.2,
                                         num_epochs=50)

Epoch 1/50
212/212 [==============================] - 5s 10ms/step - loss: 1.0556 - accuracy: 0.4282 - val_loss: 1.0400 - val_accuracy: 0.4403
Epoch 2/50
212/212 [==============================] - 2s 8ms/step - loss: 1.0371 - accuracy: 0.4431 - val_loss: 1.0265 - val_accuracy: 0.4486
Epoch 3/50
212/212 [==============================] - 2s 8ms/step - loss: 1.0266 - accuracy: 0.4508 - val_loss: 1.0159 - val_accuracy: 0.4403
Epoch 4/50
212/212 [==============================] - 2s 7ms/step - loss: 1.0106 - accuracy: 0.4573 - val_loss: 1.0186 - val_accuracy: 0.4563
Epoch 5/50
212/212 [==============================] - 2s 7ms/step - loss: 0.9911 - accuracy: 0.4750 - val_loss: 1.0010 - val_accuracy: 0.4858
Epoch 6/50
212/212 [==============================] - 1s 7ms/step - loss: 0.9727 - accuracy: 0.5108 - val_loss: 0.9660 - val_accuracy: 0.5112
Epoch 7/50
212/212 [==============================] - 2s 7ms/step - loss: 0.9551 - accuracy: 0.5151 - val_loss: 0.9635 - val_accuracy: 0.5047
Epoch

In [16]:
modelC.save("modelC.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Since Model B has best performance use that to experiment with num epochs

In [17]:
modelD , X_testD, y_testD = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.2,
                                         num_epochs=75)

Epoch 1/75
423/423 [==============================] - 8s 9ms/step - loss: 1.0572 - accuracy: 0.4204 - val_loss: 1.0254 - val_accuracy: 0.4577
Epoch 2/75
423/423 [==============================] - 3s 6ms/step - loss: 1.0287 - accuracy: 0.4544 - val_loss: 1.0011 - val_accuracy: 0.4929
Epoch 3/75
423/423 [==============================] - 3s 7ms/step - loss: 1.0055 - accuracy: 0.4764 - val_loss: 0.9683 - val_accuracy: 0.5118
Epoch 4/75
423/423 [==============================] - 3s 7ms/step - loss: 0.9798 - accuracy: 0.5055 - val_loss: 0.9494 - val_accuracy: 0.5174
Epoch 5/75
423/423 [==============================] - 3s 7ms/step - loss: 0.9486 - accuracy: 0.5321 - val_loss: 0.9362 - val_accuracy: 0.5482
Epoch 6/75
423/423 [==============================] - 3s 7ms/step - loss: 0.9164 - accuracy: 0.5525 - val_loss: 0.9220 - val_accuracy: 0.5458
Epoch 7/75
423/423 [==============================] - 3s 7ms/step - loss: 0.8832 - accuracy: 0.5780 - val_loss: 0.8623 - val_accuracy: 0.5937
Epoch 

In [18]:
modelD.save("modelD.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
modelE , X_testE, y_testE = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.2,
                                         num_epochs=100)

Epoch 1/100
423/423 [==============================] - 8s 9ms/step - loss: 1.0573 - accuracy: 0.4235 - val_loss: 1.0318 - val_accuracy: 0.4317
Epoch 2/100
423/423 [==============================] - 3s 6ms/step - loss: 1.0354 - accuracy: 0.4445 - val_loss: 1.0089 - val_accuracy: 0.4796
Epoch 3/100
423/423 [==============================] - 3s 7ms/step - loss: 1.0112 - accuracy: 0.4691 - val_loss: 0.9882 - val_accuracy: 0.4911
Epoch 4/100
423/423 [==============================] - 3s 6ms/step - loss: 0.9816 - accuracy: 0.5004 - val_loss: 0.9413 - val_accuracy: 0.5340
Epoch 5/100
423/423 [==============================] - 3s 6ms/step - loss: 0.9482 - accuracy: 0.5318 - val_loss: 0.9251 - val_accuracy: 0.5600
Epoch 6/100
423/423 [==============================] - 3s 6ms/step - loss: 0.9316 - accuracy: 0.5460 - val_loss: 0.9023 - val_accuracy: 0.5727
Epoch 7/100
423/423 [==============================] - 3s 6ms/step - loss: 0.9083 - accuracy: 0.5636 - val_loss: 0.8925 - val_accuracy: 0.5786

In [20]:
modelE.save("modelE.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Use best performance and experiment with test size

### 70-30 split

In [21]:
modelF , X_testF, y_testF = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.3,
                                         num_epochs=75)

Epoch 1/75
371/371 [==============================] - 8s 10ms/step - loss: 1.0630 - accuracy: 0.4138 - val_loss: 1.0269 - val_accuracy: 0.4762
Epoch 2/75
371/371 [==============================] - 3s 7ms/step - loss: 1.0346 - accuracy: 0.4469 - val_loss: 1.0052 - val_accuracy: 0.4890
Epoch 3/75
371/371 [==============================] - 3s 7ms/step - loss: 1.0106 - accuracy: 0.4747 - val_loss: 0.9836 - val_accuracy: 0.5218
Epoch 4/75
371/371 [==============================] - 3s 7ms/step - loss: 0.9764 - accuracy: 0.5094 - val_loss: 0.9378 - val_accuracy: 0.5606
Epoch 5/75
371/371 [==============================] - 3s 7ms/step - loss: 0.9455 - accuracy: 0.5394 - val_loss: 0.9233 - val_accuracy: 0.5535
Epoch 6/75
371/371 [==============================] - 2s 6ms/step - loss: 0.9199 - accuracy: 0.5605 - val_loss: 0.9050 - val_accuracy: 0.5681
Epoch 7/75
371/371 [==============================] - 2s 6ms/step - loss: 0.9049 - accuracy: 0.5686 - val_loss: 0.9051 - val_accuracy: 0.5694
Epoch

In [22]:
modelF.save("modelF.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### 50-50 split

In [23]:
modelG , X_testG, y_testG = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.5,
                                         num_epochs=75)

Epoch 1/75
265/265 [==============================] - 6s 8ms/step - loss: 1.0652 - accuracy: 0.4124 - val_loss: 1.0350 - val_accuracy: 0.4775
Epoch 2/75
265/265 [==============================] - 2s 6ms/step - loss: 1.0367 - accuracy: 0.4459 - val_loss: 1.0099 - val_accuracy: 0.4761
Epoch 3/75
265/265 [==============================] - 1s 5ms/step - loss: 1.0199 - accuracy: 0.4639 - val_loss: 0.9933 - val_accuracy: 0.4931
Epoch 4/75
265/265 [==============================] - 2s 6ms/step - loss: 1.0012 - accuracy: 0.4944 - val_loss: 0.9758 - val_accuracy: 0.5220
Epoch 5/75
265/265 [==============================] - 1s 5ms/step - loss: 0.9836 - accuracy: 0.5103 - val_loss: 0.9458 - val_accuracy: 0.5508
Epoch 6/75
265/265 [==============================] - 2s 6ms/step - loss: 0.9568 - accuracy: 0.5278 - val_loss: 0.9431 - val_accuracy: 0.5556
Epoch 7/75
265/265 [==============================] - 1s 5ms/step - loss: 0.9501 - accuracy: 0.5376 - val_loss: 0.9856 - val_accuracy: 0.4936
Epoch 

In [24]:
modelG.save("modelG.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score,precision_recall_fscore_support

def getMetrics(model,X_test,y_test):

    y_pred_proba = model.predict(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)
    cm = confusion_matrix(y_test.flatten(), y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=[0,1,2], yticklabels=[0,1,2])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

    # Compute precision, recall, and F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(y_test.flatten(), y_pred, average=None)

    # Print precision, recall, and F1 score for each class
    for i in range(len(precision)):
        print(f"Class {i}:")
        print(f"  Precision: {precision[i]}")
        print(f"  Recall: {recall[i]}")
        print(f"  F1 Score: {f1[i]}")
